In [1]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub, OpenAI

In [2]:
llm_repo_id = "google/flan-t5-large" 
embeding_model = "hkunlp/instructor-large"
index_file_path = 'youtube.faiss'

In [3]:
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceInstructEmbeddings(
    model_name=embeding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\Hung\anaconda3\envs\ml\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
db = FAISS.load_local(index_file_path, embeddings)

In [5]:
k = 5
question = "What is teacher's name?"

In [6]:
docs = db.similarity_search(question, k=k)
docs_page_content = " ".join([d.page_content for d in docs])

In [7]:
llm = HuggingFaceHub(
    repo_id=llm_repo_id, model_kwargs={"temperature": 0.05, "max_length": 512}
)
prompt = PromptTemplate(
    input_variables=["question", "docs"],
    template="""
You are a helpful Youtube assistanct that can answer questions about videos based on the video's transcript
        
Answer the following question: {question}
By searching the following video transcript: {docs}

Only use the factual information from the transcipt to answer the question.

If you feel like you do not have enough information to answer the question, say "I don't know".

Your answers should be detailed.
""")
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run(question=question, docs=docs_page_content)
response = response.replace("\n", "")
response

c:\Users\Hung\anaconda3\envs\ml\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


'Henry'